In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:
df = pd.read_hdf('/scratch/spigazzi/MTD/TDR/TrackPUID/samples/input_tracks_train.hd5', stop=5000000)

In [17]:
df['dx'] = abs(df['x']-df['pv4d_x'])
df['dy'] = abs(df['y']-df['pv4d_y'])
df['dz'] = abs(df['z']-df['pv4d_z'])
df['dt'] = abs(df['t0']-df['pv4d_t'])

features = [
    'pt',
    'eta',
    'phi',
    'dx',
    'dy',
    'dz',
    'dzErr',
    'dxyErr',
    'chi2',
    'ndof',
    'numberOfValidHits',
    'numberOfValidPixelBarrelHits',
    'numberOfValidPixelEndcapHits'
    ]

MTD_features = [
    'dt',
    'sigmat0',
    'hasMTD',
    'btlMatchChi2',
    'btlMatchTimeChi2',
    'etlMatchChi2',
    'etlMatchTimeChi2',
    'mtdt',
    'path_len'
]

target = 'simIsFromPV'
event_sel = (abs(df['pv4d_z']-df['genVtx_z'])<0.01) & df['pv4d_valid']

pv_pu_ratio = float((df['simIsFromPV']==1).sum())/(df['simIsFromPV']==0).sum()
df['pu_weight'] = 1.
df.loc[df[target]==0, 'pu_weight'] = pv_pu_ratio
print(pv_pu_ratio, df.loc[df['simIsFromPV']==0, 'pu_weight'].sum()/df.loc[df['simIsFromPV']==1, 'pu_weight'].sum())

X = df[features + MTD_features][event_sel]
y = df[target][event_sel]
w = df['pu_weight'][event_sel]

0.0245956228455 1.000000000032431


In [18]:
X_train, X_test, y_train, y_test, w_train, w_test = train_test_split(X, y, w, test_size=0.2, random_state=12345)

In [26]:
from sklearn.externals.joblib import Parallel, parallel_backend, register_parallel_backend
import ipyparallel as ipp
from ipyparallel.joblib import IPythonParallelBackend
global joblib_rc,joblib_view,joblib_be
joblib_rc = ipp.Client(profile='all', verbosity=20)
joblib_view = joblib_rc.load_balanced_view()
joblib_be = IPythonParallelBackend(view=joblib_view)
register_parallel_backend('ipyparallel',lambda: joblib_be, make_default=True)

In [20]:
from xgboost.sklearn import XGBClassifier 

In [21]:
clf3D = XGBClassifier(max_depth=5, n_estimators=500, nthread=8, objective='binary:logistic', subsample=0.75, learning_rate=0.1)
clf3D

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=500,
       n_jobs=1, nthread=8, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.75)

In [23]:
with parallel_backend('ipyparallel'):
    fitres3D = clf3D.fit(X_train[features], y_train, w_train, eval_set=[(X_train[features], y_train, w_train), (X_test[features], y_test, w_test)], eval_metric=['error', 'rmse', 'mae', 'logloss'])

TypeError: <lambda>() got an unexpected keyword argument 'verbosity'